In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline
import os
import keras

In [4]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1



In [5]:
print(tf.__version__)
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))

2.5.0
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
path = 'emotional database/motion capture/'
files = []
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16246519447356585223
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4848943104
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12223682259571449019
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
descriptors = pd.read_csv('Descriptors.csv', index_col=0)

In [9]:
def load_file(file):
    temp_df = pd.read_csv(file, sep=';')
    textfile = file[34:]
    output_value = descriptors.loc[descriptors['text_file']==textfile]['output'][:1].values[0]
    gender = descriptors.loc[descriptors['text_file']==textfile]['gender'][:1].values[0]
    if gender == 'F':
        temp_df['gender'] = 1
    else:
        temp_df['gender'] = 0
    temp_df.drop(['Unnamed: 152'], axis=1, inplace=True)
    return temp_df.values, output_value

In [10]:
shapes = []
Y = []
for name in files:
    data, t = load_file(name)
    shapes.append(data.shape[0])
    Y.append(t)

In [11]:
index = 0 
for element in Y:
    if element == 'Ne':
        Y[index] = 0
    elif element == 'Sa':
        Y[index] = 1
    elif element == 'Su':
        Y[index] = 2
    elif element == 'Fe':
        Y[index] = 3
    elif element == 'An':
        Y[index] = 4
    elif element == 'Di':
        Y[index] = 5
    elif element == 'Ha':
        Y[index] = 6
    index += 1

In [12]:
from tensorflow.keras.utils import to_categorical

maxlen = max(shapes)
print(maxlen)

1758


In [13]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data, temp = load_file(name)
        zeros_data = np.zeros((max(shapes)-data.shape[0], 153))
        data = np.concatenate((zeros_data, data), axis=0)
        loaded.append(data)
        shapes.append(data.shape[0])
    loaded = np.stack(loaded)
    return loaded

In [14]:
X = load_group(files)

Y = np.asarray(Y).reshape((63,1))
Y = to_categorical(Y)
print(X.shape, Y.shape)

(63, 1758, 153) (63, 7)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from functools import partial

In [18]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose = 1
    epochs = 100
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(32, input_shape=(n_timesteps, n_features)))
    #model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(trainX, trainy, epochs=epochs, verbose=verbose)
    _, accuracy = model.evaluate(testX, testy, verbose=verbose)
    return accuracy

In [19]:
evaluate_model(X_train, Y_train, X_test, Y_test)

Epoch 1/100
2/2 [==============================] - 15s 84ms/step - loss: 2.0222 - accuracy: 0.1607
Epoch 2/100
2/2 [==============================] - 0s 68ms/step - loss: 2.0156 - accuracy: 0.1607
Epoch 3/100
2/2 [==============================] - 0s 71ms/step - loss: 2.0093 - accuracy: 0.1607
Epoch 4/100
2/2 [==============================] - 0s 69ms/step - loss: 2.0039 - accuracy: 0.1607
Epoch 5/100
2/2 [==============================] - 0s 68ms/step - loss: 1.9998 - accuracy: 0.1607
Epoch 6/100
2/2 [==============================] - 0s 71ms/step - loss: 1.9945 - accuracy: 0.1607
Epoch 7/100
2/2 [==============================] - 0s 69ms/step - loss: 1.9905 - accuracy: 0.1607
Epoch 8/100
2/2 [==============================] - 0s 69ms/step - loss: 1.9858 - accuracy: 0.1607
Epoch 9/100
2/2 [==============================] - 0s 74ms/step - loss: 1.9817 - accuracy: 0.1607
Epoch 10/100
2/2 [==============================] - 0s 79ms/step - loss: 1.9780 - accuracy: 0.1607
Epoch 11/100
2/2 [

0.0

In [17]:
df = pd.DataFrame()
for file in files:
    temp_df = pd.read_csv(file, sep=';')
    textfile = file[34:]
    output_value = descriptors.loc[descriptors['text_file']==textfile]['output'][:1].values[0]
    video_name = descriptors.loc[descriptors['text_file']==textfile]['video_name'][:1].values[0]
    gender = descriptors.loc[descriptors['text_file']==textfile]['gender'][:1].values[0]
    temp_df['textfile'] = textfile
    temp_df['output'] = output_value
    temp_df['video_name'] = video_name
    temp_df['gender'] = gender
    df = df.append(temp_df, sort=False)

In [18]:
df.drop(columns=['Unnamed: 152'], inplace=True)

In [19]:
df.to_csv('Motion_capture.csv')